In [ ]:
import mysql.connector
import pandas as pd

class RentalDBManager :

    def __init__(self, host, port, user, password, database):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.db = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=database
        )
        self.cursor = self.db.cursor(buffered=True)
        print("DB connected")

    def close(self):
        if self.cursor:
            self.cursor.close()
        if self.db:
            self.db.close()
        print("DB disconnected")

    def insert(self, rental_datetime, return_datetime, rental_station_id, return_station_id, distance, birth_year):

        sql = """
        INSERT INTO rental (
            rental_datetime, return_datetime,
            rental_station_id, return_station_id,
            distance, birth_year
        ) VALUES (%s, %s, %s, %s, %s, %s)
        """
        self.cursor.execute(sql, (rental_datetime, return_datetime, rental_station_id, return_station_id, distance, birth_year))
        self.db.commit()
        print("INSERTED")
        return True

    def select_all(self, table):
        sql = f"SELECT * FROM {table}"
        self.cursor.execute(sql)
        result = self.cursor.fetchall()
        columns = [col[0] for col in self.cursor.description]
        df = pd.DataFrame(result, columns=columns).reset_index(drop=True)
        return df

    def execute(self, sql):
        self.cursor.execute(sql)
        self.db.commit()
        print("------------------------------------------")
        print("EXECUTED")
        if self.cursor.description is not None:
            result = self.cursor.fetchall()
            print(result)
        return True

dbm = RentalDBManager(
    host="jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port= 3306,
    user="root",
    password="jinhyuk2dacibul",
    database="project1"
)

DB connected


In [ ]:
import pandas as pd
import glob
import os

# 경로 지정
folder_path = '/content/drive/MyDrive/EDA Project/대여이력/'
chunksize = 100000
all_chunks = []

# 모든 CSV 파일 경로 가져오기
file_list = glob.glob(os.path.join(folder_path, '*.csv'))

# 파일별로 chunk 단위로 읽어서 리스트에 추가
for file in file_list:
    print(f"📂 {os.path.basename(file)} 읽는 중...")
    for chunk in pd.read_csv(file, chunksize=chunksize, encoding='cp949', low_memory=False):
        all_chunks.append(chunk)

# 하나의 DataFrame으로 합치기
df12 = pd.concat(all_chunks, ignore_index=True)
print("✅ 모든 파일 병합 완료:", df12.shape)

📂 서울특별시 공공자전거 대여이력 정보_2404.csv 읽는 중...
📂 서울특별시 공공자전거 대여이력 정보_2403.csv 읽는 중...
📂 서울특별시 공공자전거 대여이력 정보_2405.csv 읽는 중...
📂 서울특별시 공공자전거 대여이력 정보_2409.csv 읽는 중...
📂 서울특별시 공공자전거 대여이력 정보_2412.csv 읽는 중...
📂 서울특별시 공공자전거 대여이력 정보_2411.csv 읽는 중...
📂 서울특별시 공공자전거 대여이력 정보_2410.csv 읽는 중...
📂 서울특별시 공공자전거 대여이력 정보_2402.csv 읽는 중...
📂 서울특별시 공공자전거 대여이력 정보_2408.csv 읽는 중...
📂 서울특별시 공공자전거 대여이력 정보_2407.csv 읽는 중...
📂 서울특별시 공공자전거 대여이력 정보_2406.csv 읽는 중...
📂 서울특별시 공공자전거 대여이력 정보_2401.csv 읽는 중...
✅ 모든 파일 병합 완료: (44543263, 17)


In [ ]:
df = df12[['대여일시', '반납일시', '대여대여소ID', '반납대여소ID', '이용거리(M)', '생년']]
df['생년'] = df['생년'].replace('\\N', pd.NA)
df = df.where(pd.notnull(df), None)

<ipython-input-6-79d9ac35e4df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['생년'] = df['생년'].replace('\\N', pd.NA)


In [ ]:
from tqdm import tqdm
from datetime import datetime

# 전처리: NaN → None
df = df.where(pd.notnull(df), None)

# 날짜 변환
df['대여일시'] = pd.to_datetime(df['대여일시'], errors='coerce')
df['반납일시'] = pd.to_datetime(df['반납일시'], errors='coerce')

# 리스트로 변환 (진행률 표시)
values = []
for i, row in tqdm(df.iterrows(), total=len(df), desc="데이터 변환 중"):
    rental_time = row['대여일시'].to_pydatetime() if isinstance(row['대여일시'], pd.Timestamp) else None
    return_time = row['반납일시'].to_pydatetime() if isinstance(row['반납일시'], pd.Timestamp) else None

    values.append((
        rental_time,
        return_time,
        row['대여대여소ID'],
        row['반납대여소ID'],
        float(row['이용거리(M)']) if row['이용거리(M)'] is not None else None,
        int(row['생년']) if row['생년'] is not None else None,
    ))

    if i < 5:
        print(values[-1])

데이터 변환 중:   0%|          | 2767/44543263 [02:24<370:59:03, 33.35it/s]

(datetime.datetime(2024, 4, 1, 0, 0, 2), datetime.datetime(2024, 4, 1, 0, 0, 11), 'ST-2827', 'ST-2827', 0.0, 2004)
(datetime.datetime(2024, 4, 1, 0, 0, 38), datetime.datetime(2024, 4, 1, 0, 0, 48), 'ST-492', 'ST-492', 0.0, 2003)
(datetime.datetime(2024, 4, 1, 0, 0, 38), datetime.datetime(2024, 4, 1, 0, 0, 55), 'ST-1105', 'ST-1105', 0.0, 2001)
(datetime.datetime(2024, 4, 1, 0, 1, 44), datetime.datetime(2024, 4, 1, 0, 1, 52), 'ST-112', 'ST-112', 0.0, 1998)
(datetime.datetime(2024, 4, 1, 0, 0, 5), datetime.datetime(2024, 4, 1, 0, 1, 56), 'ST-1435', 'ST-2513', 260.0, 1999)


데이터 변환 중: 100%|██████████| 44543263/44543263 [57:04<00:00, 13005.86it/s]


In [ ]:
sql = """
    INSERT INTO rental (
        rental_datetime, return_datetime,
        rental_station_id, return_station_id, distance,
        birth_year
    ) VALUES (%s, %s, %s, %s, %s, %s)
"""

batch_size = 10000

for i in range(0, len(values), batch_size):
    batch = values[i:i+batch_size]
    dbm.cursor.executemany(sql, batch)
    dbm.db.commit()
    print(f"✅ [{i} ~ {i+len(batch)}] INSERT 완료")

✅ [0 ~ 10000] INSERT 완료
✅ [10000 ~ 20000] INSERT 완료
✅ [20000 ~ 30000] INSERT 완료
✅ [30000 ~ 40000] INSERT 완료
✅ [40000 ~ 50000] INSERT 완료
✅ [50000 ~ 60000] INSERT 완료
✅ [60000 ~ 70000] INSERT 완료
✅ [70000 ~ 80000] INSERT 완료
✅ [80000 ~ 90000] INSERT 완료
✅ [90000 ~ 100000] INSERT 완료
✅ [100000 ~ 110000] INSERT 완료
✅ [110000 ~ 120000] INSERT 완료
✅ [120000 ~ 130000] INSERT 완료
✅ [130000 ~ 140000] INSERT 완료
✅ [140000 ~ 150000] INSERT 완료
✅ [150000 ~ 160000] INSERT 완료
✅ [160000 ~ 170000] INSERT 완료
✅ [170000 ~ 180000] INSERT 완료
✅ [180000 ~ 190000] INSERT 완료
✅ [190000 ~ 200000] INSERT 완료
✅ [200000 ~ 210000] INSERT 완료
✅ [210000 ~ 220000] INSERT 완료
✅ [220000 ~ 230000] INSERT 완료
✅ [230000 ~ 240000] INSERT 완료
✅ [240000 ~ 250000] INSERT 완료
✅ [250000 ~ 260000] INSERT 완료
✅ [260000 ~ 270000] INSERT 완료
✅ [270000 ~ 280000] INSERT 완료
✅ [280000 ~ 290000] INSERT 완료
✅ [290000 ~ 300000] INSERT 완료
✅ [300000 ~ 310000] INSERT 완료
✅ [310000 ~ 320000] INSERT 완료
✅ [320000 ~ 330000] INSERT 완료
✅ [330000 ~ 340000] INSERT 완료
✅ [